### Preprocesess

In [1]:
path_to_MyModule = '..'

import sys
sys.path.insert(0, path_to_MyModule) 

import pandas as pd
from scipy.stats import randint
from scipy.stats import uniform
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from MyModule.GeneralFunctions import *
from MyModule.SummarizationFunctions import *
from MyModule.SamplingFunctions import *

In [2]:
df = pd.read_excel('..\datos.xlsx')[['ID','texto','desafio','rate']]

In [3]:
# Limpieza
df.drop_duplicates(subset='texto', inplace=True)

# Quitando texto de mas en columna "desafio"
df['desafio'] = df['desafio'].apply(lambda x: re.findall('[0-9]+', x)[0])

# A str
df['texto'] = df['texto'].astype(str)

df.reset_index(inplace=True, drop=True)

In [3]:
# Load dfs
df_predic = pd.read_csv('df_predic.csv')[['ID', 'desafio', 'target', 'doc_pos', 'doc_neg', 'doc_neu', 'doc_len']]

In [4]:
df_semantic = pd.read_csv('../Semantic_Analysis/Datasets/df_semantic.csv')

In [5]:
df_predic = df_predic.join(df_semantic, how='left')
df_predic

,ID,desafio,target,doc_pos,doc_neg,doc_neu,doc_len,sem_variability,sem_granularity
0,749,15,0.00,0.168139,0.318913,0.512949,1,NaN,0.000000
1,748,13,0.00,0.009585,0.827293,0.163122,13,0.005862,9.500000
2,747,14,0.00,0.014568,0.546193,0.439239,12,0.016918,4.500000
3,746,12,0.00,0.565521,0.023063,0.411416,23,0.053762,5.166667
4,745,16,0.00,0.028001,0.495369,0.476630,31,0.008742,6.750000
...,...,...,...,...,...,...,...,...,...
500,165,13,-0.33,0.070960,0.601193,0.327847,16,0.051103,4.500000
501,164,15,-0.40,0.485169,0.089189,0.425642,23,0.018819,3.333333
502,163,13,0.11,0.004152,0.966732,0.029116,26,0.022065,6.000000
503,162,15,-0.20,0.630141,0.053619,0.316240,3,NaN,9.000000


# Modeling

In [6]:
semilla = 2023

## Scaling

In [7]:
#Importamos StandardScaler de la libreria sklear
from sklearn.preprocessing import StandardScaler

# Instanciamos la funcion para escalar
scaler = StandardScaler()

#Escalamos las variables
scaled = scaler.fit_transform(df_predic.drop(['ID','desafio'], axis=1))
df_predic_scaled = pd.DataFrame(scaled, columns = df_predic.columns[2:])
df_predic_scaled.fillna(0, inplace=True)

## Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

# Partición en train y test
y = df_predic_scaled.loc[:, df_predic_scaled.columns == 'target']['target'].values.tolist()
X = df_predic_scaled.loc[:, df_predic_scaled.columns != 'target']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=semilla)

# Split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=semilla)

### Linear Regression

In [9]:
# instancia del modelo
linearRegr = LinearRegression()

# entrenamiento
linearRegr.fit(X_train, y_train)

# basic performance
y_pred = linearRegr.predict(X_val)

In [10]:
# Claramente las clases estan desbalanceadas
print_performance(y_val, y_pred)

R-squared score: 0.0057253299567038285
Mean Squared Error: 0.7858695615271343
Mean Absolute Error: 0.7189707112664292
